# Stock trading agent

## Data
We have a folder from https://www.kaggle.com/datasets/camnugent/sandp500 which includes the prices every market open day from 08-02-2013 to 08-02-2018 fro over 500 companies. These are companies in the S&P 500 and are the top 500 markets in the US, furthermore they represent the shares of the most commonly traded companies in the stock market.

The data has 5 columns for each company in its own file named by its stock name:
- Date: yy-mm-dd
- Open: price of stock at market open
- High: Highest price of stock in day
- Low: Lowest price of stock in day
- Close: Price of closing
- Volume: How many stocks traded that day for that company
- Name: The ticker name of the company in the stock market

In [2]:
from pathlib import Path

def count_files(directory_path):
    path = Path(directory_path)
    count = len([p for p in path.iterdir() if p.is_file()])
    return count

directory = 'individual_stocks_5yr'
file_count = count_files(directory)
print(f"Number of files in '{directory}': {file_count}")

Number of files in 'individual_stocks_5yr': 505


### Reading the data
Here we read the csv files for the first 100 companies and store the csv file in a panda dataframe and then store the 100 dataframes in a simple list.

In [ ]:
# Schema: date | open | high | low | close | volume | name

# import os

# folder_path = 'individual_stocks_5yr'

# entries = os.listdir(folder_path)

# import pandas as pd

# stock_prices = []

# for i in range(100):
#     filename = folder_path + "/" + entries[i]
#     try:
#         df = pd.read_csv(filename)
#         stock_prices.append(df)
#     except:
#         print("Error: Error reading file (" + filename + ")")

# # A vector with the days for conveinience later
# calendar = stock_prices[0]['date']

# # print(len(stock_prices))


0       2013-02-08
1       2013-02-11
2       2013-02-12
3       2013-02-13
4       2013-02-14
           ...    
1254    2018-02-01
1255    2018-02-02
1256    2018-02-05
1257    2018-02-06
1258    2018-02-07
Name: date, Length: 1259, dtype: object


In [ ]:
# Schema: date | open | high | low | close | volume | name

from pathlib import Path
import pandas as pd

def read_data(folder_path, amount = None):
    path = Path(folder_path)
    files = []

    for f in path.iterdir():
        if f.is_file() and f.suffix.lower() == '.csv':
            filename = f
            try:
                df = pd.read_csv(filename)
                files.append(df)
                if  amount:
                    amount -= 1
                    if amount == 0:
                        return files
            except:
                print("Error: Error reading file (" + filename + ")")

    return files

def inspect_data(comp_list):
    def check_for_na(comp_df):
        total_nas = comp_df.isna().sum().sum()

        if total_nas != 0:
            print("Null values found. File: %s. Count: %s", comp_df.iat[0][6], total_nas)
    
    def check_length(comp_list):
        deffault = comp_list[0].shape
        uniform = True

        for i, df in enumerate(comp_list):
            if df.shape != deffault:
                print(f"Unequal size detected. Deffault: {deffault}. Detected size: {df.shape}")
                uniform = False
        if uniform:
            print("All dfs same shape")            

    def check_dates(comp_list):
        calendar = comp_list[0]['date']
        uniform = True

        for i, df in enumerate(comp_list):
            if df['date'] != calendar:
                print(f"Unequal dates detected for file {df.iat[0][6]}")
                unifrom = False
        if uniform:
            print('All dates are equal')
    
    if not comp_list or len(comp_list) <= 1:
            return
    


In [17]:
num_comp = 1
num_days = 30

n = 5
for i in range(num_days):
    momentums = {}

    for j in range(num_comp):
        if i >= 5:
            comp_df = stock_prices[j]

            price_n_ago = comp_df.iat[i - 5, 4]

            curr_price = comp_df.iat[i, 4]
            
            momentums[comp_df.iat[i, 6]] = (curr_price - price_n_ago) / price_n_ago
    
    print(momentums)


{}
{}
{}
{}
{}
{'CMS': 0.004286827747466852}
{'CMS': 0.016361511492013953}
{'CMS': 0.008126934984520157}
{'CMS': 0.010054137664346543}
{'CMS': 0.025242718446601888}
{'CMS': 0.013969732246798582}
{'CMS': 0.00344959754695285}
{'CMS': 0.01266794625719763}
{'CMS': 0.01875957120980086}
{'CMS': 0.0018939393939394209}
{'CMS': 0.02870264064293915}
{'CMS': 0.031321619556913684}
{'CMS': 0.019332827899924247}
{'CMS': 0.009019165727170311}
{'CMS': 0.023818525519848734}
{'CMS': 0.011160714285714312}
{'CMS': -0.0011111111111111532}
{'CMS': 0.008181480104127887}
{'CMS': 0.018621973929236497}
{'CMS': 0.013293943870014882}
{'CMS': 0.0007358351729212499}
{'CMS': 0.004449388209121283}
{'CMS': 0.00590188122464036}
{'CMS': -0.008043875685557676}
{'CMS': -0.00400874635568524}


In [ ]:
def momentum(comp_df, curr, n):
    price_n_ago = comp_df.iat[curr - n, 4]

    curr_price = comp_df.iat[curr, 4]
            
    return (curr_price - price_n_ago) / price_n_ago

def get_momentums(comp_list, curr):
    momentums = {}
    n = 5
    if curr < 5:
        raise ValueError("Error: Current date is less than n!")
    

    for comp in comp_list:
        momentums[comp.iat[0][6]] = momentum(comp, curr, n)

    return momentums

